In [25]:
import cv2 as cv
import random
import os
import math
from tqdm import tqdm

In [26]:
# This is the path of the folder that contains all the videos where to extract the training/test frames
SOURCE = os.getenv("HOME") + "/shared"
# The folder where to store the frames.
DESTINATION = os.getenv("HOME") + "/Projects/CV/img_denoise/data"
# Width for the output frames
RES_W = 256
# Height for the output frames
RES_H = 144
# The subfolder for training dataset
TRAIN = "train"
# The subfolder for tests dataset
TEST =  "test"
# Number of training data
N_TRAIN=1000
# Number of testing data
N_TEST=40
# File format
EXT="png"

In [ ]:
# Some preprocessing...
TRAIN = DESTINATION + "/" + TRAIN
TEST = DESTINATION +  "/" + TEST
EXT= "." + EXT
ASP_W = RES_W / math.gcd(RES_W, RES_H)
ASP_H = RES_H / math.gcd(RES_W, RES_H)

# Generation of the destination folders if they don't already exists
os.makedirs(TRAIN, exist_ok = True)
os.makedirs(TEST, exist_ok = True)

In [28]:
# video : The video file where to extract the frame
# position [0, 1]: Position of the frame in the video, where 0 is the beginning and 1 is the end
def extract_frame(video, position : float):
    
    # Validation for the position
    if position < 0 or position > 1:
        print("The position must between 0 and 1!")
        return
    
    frames = int(video.get(cv.CAP_PROP_FRAME_COUNT))
    curr_frame = int(position * frames)
    
    if not video.set(cv.CAP_PROP_POS_FRAMES, curr_frame):
        print("Cannot get to the frame!")
        return
    
    ret, img = video.read()
    # Computing the aspect ratio of the image.
    (h, w) = img.shape[:2]
    asp_w = w / math.gcd(w, h)
    asp_h = h / math.gcd(w, h)
    
    # Crop the image if the aspect ratio don't match
    # Ex: image 4:3 asp, while the final image must be 16:9
    if not (asp_h == ASP_H and asp_w == ASP_W):
        offset = int((h - RES_H) // 2)
        new_h = int((w * ASP_H) // ASP_W)
        img = img[offset:offset + new_h, :]
    img = cv.resize(img, dsize = (RES_W, RES_H), interpolation = cv.INTER_AREA)
    return img
    
    

In [29]:
n_train = N_TRAIN // len(os.listdir(SOURCE))
n_test = N_TEST // len(os.listdir(SOURCE))

j = 0
for video_file in tqdm(os.listdir(SOURCE)):
    video = cv.VideoCapture(SOURCE + "/" + video_file)
    
    if not video.isOpened():
        print("Oh Noo! The file cannot be opened : " + video_file)
        continue
    
    for i in range(n_train):
        # Avoiding the initial logos and the credit scene
        rand = random.uniform(0.2, 0.8)
        r_frame = extract_frame(video, rand)
        cv.imwrite(TRAIN + "/" + str(j * n_train + i) + EXT, r_frame)
        
    for i in range(n_test):
        rand = random.uniform(0.2, 0.8)
        r_frame = extract_frame(video, rand)
        cv.imwrite(TEST + "/" + str(j * n_test + i) + EXT, r_frame)
    
    j += 1
        
    

100%|██████████| 20/20 [01:32<00:00,  4.63s/it]
